<a href="https://colab.research.google.com/github/rajdas2001/Fine-tuning-LLM-on-Custom-Dataset/blob/main/Fine_tuning_LLama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

df = pd.read_csv("combined_data.csv",encoding='unicode_escape')
df.head()

,prompt,response,prompt_word_count,response_word_count
0,Who is Raj Das?,Raj Das is a Data Scientist.,4,6
1,Who built you?,I was built by Raj Das.,3,6
2,What is psoriasis and what are its common symp...,Psoriasis is a chronic autoimmune condition th...,9,67
3,What is the etiology of acne?,Acne is primarily caused by the overproduction...,6,64
4,What are the recommended medications for atopi...,There are several medications available for th...,8,83


In [3]:
!nvidia-smi

Mon Apr 15 19:06:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00


In [5]:
import os
import random
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import pipeline

In [6]:
system_message  = "Hello! I'm here to provide concise information about various skin diseases, including their causes, symptoms, treatments, and recommended medications. How can I assist you today?"
print(system_message)

Hello! I'm here to provide concise information about various skin diseases, including their causes, symptoms, treatments, and recommended medications. How can I assist you today?


In [7]:
df = df.drop("prompt_word_count", axis = 1)
df = df.drop("response_word_count", axis = 1)
df.head()

,prompt,response
0,Who is Raj Das?,Raj Das is a Data Scientist.
1,Who built you?,I was built by Raj Das.
2,What is psoriasis and what are its common symp...,Psoriasis is a chronic autoimmune condition th...
3,What is the etiology of acne?,Acne is primarily caused by the overproduction...
4,What are the recommended medications for atopi...,There are several medications available for th...


In [8]:
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

In [9]:
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

In [10]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "/content/train.jsonl"

new_model = "Medi-llama-2-7b-custom1000"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

output_dir = "./results"

num_train_epochs = 5

fp16 = False
bf16 = False

per_device_train_batch_size = 4

per_device_eval_batch_size = 4
gradient_accumulation_steps = 1

gradient_checkpointing = True

max_grad_norm = 0.3


learning_rate = 2e-4
weight_decay = 0.001

optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"

max_steps = -1

warmup_ratio = 0.03

group_by_length = True

save_steps = 0

logging_steps = 25

max_seq_length = None
packing = False

device_map = {"": 0} ## LOAD THE ENTIRE MODLE ON THE GPU

In [11]:
# Load datasets
train_dataset = load_dataset('json', data_files='/content/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <>\n{system_message.strip()}\n<>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <>\n{system_message.strip()}\n<>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1316 [00:00<?, ? examples/s]

Map:   0%|          | 0/146 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1316 [00:00<?, ? examples/s]

Map:   0%|          | 0/146 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.611800
50,1.169100
75,0.816300
100,0.708300
125,0.785300
150,0.663500
175,0.745400
200,0.670900
225,0.738900
250,0.655100


In [12]:
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <>\n{system_message}\n<>\n\nWho is Raj Das? [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(prompt)
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] <>
Hello! I'm here to provide concise information about various skin diseases, including their causes, symptoms, treatments, and recommended medications. How can I assist you today?
<>

Who is Raj Das? [/INST] Raj Das is a Data Scientist at Skin Care Advisor. He has a strong background in computer science and a passion for healthcare, particularly skin health. He is responsible for developing and maintaining the algorithms that power the site, ensuring that the information provided is accurate and up-to-date. He also contributes to the development of new features and functionalities of the site. Raj is a key member of the team and plays a crucial role in the success of Skin Care Advisor.

In his free time, Raj enjoys reading about the latest advancements in technology and its applications in healthcare. He also enjoys hiking and traveling to new places. Raj is based in our headquarters in New York City.


In [13]:
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <>\n{system_message}\n<>\n\nGive Me Medication for diaper Rash? [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(prompt)
print(result[0]['generated_text'])

[INST] <>
Hello! I'm here to provide concise information about various skin diseases, including their causes, symptoms, treatments, and recommended medications. How can I assist you today?
<>

Give Me Medication for diaper Rash? [/INST] The treatment for diaper rash typically involves keeping the skin clean and moisturized. Here are some medications that can be used:

1. Zinc oxide cream or ointment: This is often the first line of treatment for mild cases of diaper rash. It helps to protect the skin and heal any minor irritation.

2. Hydrocortisone cream: This is a stronger steroid cream that can be used for more severe cases of diaper rash. It can help to reduce inflammation and heal the skin faster.

3. Antibiotic creams or ointments: If the diaper rash is caused by a bacterial infection, an antibiotic cream or ointment may be prescribed.

4. Pain relievers: Over-the-counter pain relievers like ibuprofen or acetaminophen can help to reduce discomfort.

Remember, it's important to ch

In [14]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): 

In [15]:
prompt = f"[INST] <>\n{system_message}\n<>\n\nWhat is siroline syrup used for? [/INST]"
num_new_tokens = 200

num_prompt_tokens = len(tokenizer(prompt)['input_ids'])
max_length = num_prompt_tokens + num_new_tokens

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
result = gen(prompt)
print(result[0]['generated_text'].replace(prompt, ''))

 Siroline syrup is primarily used to treat fungal infections of the skin, such as athlete's foot, jock itch, and ringworm. It's also sometimes used to treat fungal nail infections. The active ingredient in Siroline, Miconazole, works by killing the fungi causing the infection. It's important to use the syrup as directed by your healthcare provider to ensure the infection is fully treated and to prevent recurrence. If you have any concerns or questions, you should consult with your healthcare provider.

Remember, Siroline is a prescription medication and should only be taken under the supervision of a healthcare provider. It's not intended for use in children or infants.

Also, it's important to keep the skin area clean and dry to prevent fungal infections. If you have a fungal infection,
